# Ping Pong Catching

### Translational Projectile Matching without SNOPT

_Final Project for 6.843, Fall 2021_

_Viraj Parimi, Cameron Pittman_

In [ ]:
import time
import pydot
import numpy as np
from manipulation.meshcat_cpp_utils import (
    StartMeshcat
)
from pydrake.systems.meshcat_visualizer import MeshcatVisualizer
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess

from pydrake.all import (
    BasicVector, DiagramBuilder, Integrator, JacobianWrtVariable, LeafSystem, MeshcatVisualizerCpp,
    MeshcatVisualizerParams, Simulator
)

from iiwa_paddle_station import MakeIiwaPaddleStation
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=[])

In [ ]:
meshcat = StartMeshcat()

t=2021-12-08T02:01:15+0000 lvl=warn msg="can't bind default web address, trying alternatives" obj=web addr=127.0.0.1:4040


### Define Controller

In [ ]:
class PseudoInverseController(LeafSystem):
    """
    Based on the PseudoInverseController from pset3/robot_painter
    """

    def __init__(self, plant):

        LeafSystem.__init__(self)
        
        self.plant = plant
        self.W = plant.world_frame()
        self.plant_context = plant.CreateDefaultContext()
        self.iiwa = plant.GetModelInstanceByName("iiwa7")
        self.P = plant.GetBodyByName("base_link").body_frame()
        self.iiwa_start = self.plant.GetJointByName("iiwa_joint_1").velocity_start()
        self.iiwa_end = self.plant.GetJointByName("iiwa_joint_7").velocity_start()

        # Inputs
        self.q_port = self.DeclareVectorInputPort("iiwa_pos_measured", BasicVector(7))
        self.w_P_port = self.DeclareVectorInputPort("paddle_desired_angular_velocity", BasicVector(3))
        self.v_P_port = self.DeclareVectorInputPort("paddle_desired_velocity", BasicVector(3))

        # Outputs
        self.DeclareVectorOutputPort("iiwa_velocity", BasicVector(7), self.CalcOutput)

    def CalcOutput(self, context, output):

        q = self.q_port.Eval(context)
        w_P = self.w_P_port.Eval(context)
        v_P = self.v_P_port.Eval(context)
        self.plant.SetPositions(self.plant_context, self.iiwa, q)
        J_P = self.plant.CalcJacobianSpatialVelocity(
            self.plant_context,
            JacobianWrtVariable.kV,
            self.P,
            [0, 0, 0],
            self.W,
            self.W,
        )
        J_P = J_P[:, self.iiwa_start : self.iiwa_end + 1]
        V_P = np.hstack([w_P, v_P])
        v = np.linalg.pinv(J_P).dot(V_P)
        
        output.SetFromVector(v)


### Constants

In [ ]:
iiwa_q0 = np.array([-1.57, 0, 0, -np.pi/2, 0, 0, 0])
paddle_state_pos = np.array([0.07659419, -0.75730758, 0.63302061])

g = -9.81
height = 1
offset = 0.5
CATCH_HEIGHT = 0.55
simulation_time = 3
e_z = np.array([0, 0, 1], dtype = float)

### Define Initial States

In [ ]:
# Experiment 1 - drop on top of the paddle
ball_q0_exp1 = [1, 1, 1, 1, paddle_state_pos[0], paddle_state_pos[1], paddle_state_pos[2] + height, 0, 0, 0, 0, 0, 0]

# Experiment 2 - drop from (+ve x) offset with (-ve x) velocity
ball_q0_exp2 = [1, 1, 1, 1, paddle_state_pos[0] + offset, paddle_state_pos[1], paddle_state_pos[2] + height, 0, 0, 0, -1, 0, 0]

# Experiment 3 - drop from (-ve x) offset with (+ve x) velocity
ball_q0_exp3 = [1, 1, 1, 1, paddle_state_pos[0] - offset, paddle_state_pos[1], paddle_state_pos[2] + height, 0, 0, 0, 1, 0, 0]

# Experiment 4 - drop from (-ve y) offset with (+ve y) velocity
ball_q0_exp4 = [1, 1, 1, 1, paddle_state_pos[0], paddle_state_pos[1] - offset, paddle_state_pos[2] + height, 0, 0, 0, 0, 1, 0]

# Experiment 5 - drop from (+ve y) offset with (-ve y) velocity
ball_q0_exp5 = [1, 1, 1, 1, paddle_state_pos[0], paddle_state_pos[1] + offset, paddle_state_pos[2] + height, 0, 0, 0, 0, -1, 0]

# Experiment 6 - drop from (-ve x, -ve y) offset with (+ve x, +ve y) velocity
ball_q0_exp6 = [1, 1, 1, 1, paddle_state_pos[0] - offset, paddle_state_pos[1] - offset, paddle_state_pos[2] + height, 0, 0, 0, 1, 1, 0]

# Experiment 7 - drop from (-ve x, +ve y) offset with (+ve x, -ve y) velocity
ball_q0_exp7 = [1, 1, 1, 1, paddle_state_pos[0] - offset, paddle_state_pos[1] + offset, paddle_state_pos[2] + height, 0, 0, 1, -1, 0, 0]

# Experiment 8 - drop from (+ve x, -ve y) offset with (-ve x, +ve y) velocity
ball_q0_exp8 = [1, 1, 1, 1, paddle_state_pos[0] + offset, paddle_state_pos[1] - offset, paddle_state_pos[2] + height, 0, 0, 0, -1, 1, 0]

# Experiment 9 - drop from (+ve x, +ve y) offset with (-ve x, -ve y) velocity
ball_q0_exp9 = [1, 1, 1, 1, paddle_state_pos[0] + offset, paddle_state_pos[1] + offset, paddle_state_pos[2] + height, 0, 0, 0, -1, -1, 0]

scenarios = [ball_q0_exp1, ball_q0_exp2, ball_q0_exp3, ball_q0_exp4, ball_q0_exp5, ball_q0_exp6, ball_q0_exp7, ball_q0_exp8, ball_q0_exp9]

### Utility Functions

In [ ]:
def ball_p_from_state(pose):
    """
    Get x, y, z from ball_state

    Params:
        BasicVector(13)
    Returns:
        array[3]
    """
    return pose[4:7]


def ball_v_from_state(pose):
    """
    Get translational velocity from ball_state

    Params:
        BasicVector(13)
    Returns:
        array[3]
    """
    return pose[-3:]


def get_mode(p_ball, v_ball, p_paddle, atol = 1e-03):
    """
    Determine current paddle mode

    Output is one of: "rising", "falling", "contact"
    """
    b_z = p_ball[2]
    vz = v_ball[2]
    p_z = p_paddle[2]

    if np.isclose(b_z, p_z, atol=atol):
        return "contact"
    else:
        return "rising" if vz >= 0 else "falling"

### Define the Catcher class

In [ ]:
class Catcher(LeafSystem):
    """
    Catcher class that computes the trajectory of ball and outputs desired velocity estimates of the
    paddle to reach the ball at a fixed height. This approach does not compute the desired angular 
    velocity estimates of the paddle.
    """

    def __init__(self, plant):

        LeafSystem.__init__(self)
        
        self.plant = plant
        self.t_catch = None
        self.first_fall = True
        self.W = plant.world_frame()
        self.ball = plant.GetBodyByName("ball")
        self.P = plant.GetBodyByName("base_link")
        self.iiwa = plant.GetModelInstanceByName("iiwa7")
        self.plant_context = plant.CreateDefaultContext()

        # Inputs
        self.DeclareVectorInputPort("ball_state", BasicVector(13))
        self.DeclareVectorInputPort("iiwa_pos_measured", BasicVector(7))
        self.DeclareVectorInputPort("iiwa_velocity_estimated", BasicVector(7))
        
        # Outputs
        self.DeclareVectorOutputPort("paddle_desired_velocity", BasicVector(3), self.CalcDesVelOutput)
        self.DeclareVectorOutputPort("paddle_desired_angular_velocity", BasicVector(3), self.CalcDesOrientOutput)

    def compute_time_of_impact(self, p_bz, v_bz, catch_height):
        g_pos = abs(g)
        return (1. / g_pos) * (v_bz + np.sqrt(v_bz**2 + 2 * g_pos * abs(p_bz - catch_height)))

    def CalcDesOrientOutput(self, context, output):
        # Does not compute the desired orientation of the paddle
        output.SetFromVector(np.zeros(3))

    def CalcDesVelOutput(self, context, output):

        q = self.GetInputPort("iiwa_pos_measured").Eval(context)
        v = self.GetInputPort("iiwa_velocity_estimated").Eval(context)
        self.plant.SetPositionsAndVelocities(self.plant_context, self.iiwa, np.hstack([q, v]))
        
        state_Ball = np.array(self.GetInputPort("ball_state").Eval(context))
        p_Ball = ball_p_from_state(state_Ball)
        v_Ball = ball_v_from_state(state_Ball)
         
        p_Paddle = self.plant.EvalBodyPoseInWorld(self.plant_context, self.P).translation()
        v_Paddle = self.plant.EvalBodySpatialVelocityInWorld(self.plant_context, self.P).translational()

        time_of_contact = self.compute_time_of_impact(p_Ball[2], v_Ball[2], CATCH_HEIGHT)

        pred_position_of_ball_at_impact = p_Ball + v_Ball * time_of_contact + (g / 2) * e_z * time_of_contact ** 2
        pred_velocity_of_ball_at_impact = v_Ball + g * e_z * time_of_contact
        
        d_paddle_to_contact = p_Paddle[2] - CATCH_HEIGHT
        time_P_to_reach_catch_height = abs(d_paddle_to_contact / pred_velocity_of_ball_at_impact[2])

        mode = get_mode(p_Ball, v_Ball, p_Paddle, atol=1e-02)
        
        if mode == "falling" and self.first_fall:
            v_P_desired = pred_velocity_of_ball_at_impact * (time_P_to_reach_catch_height / time_of_contact)
        else:
            self.first_fall = False
            v_P_desired = np.zeros(3)
        
        output.SetFromVector(v_P_desired)


### Experimental Setup

In [ ]:
class Demo:
    def __init__(self, time_step=0.001, exp_idx = 0):
        """
        Robotic Kuka iiwa with paddle end effector

        Args:
            time_step (float, optional): time step for internal manipulation station controller. Defaults to 0.001.
            exp_idx (int, optional): experiment index to run. Defaults to 0.
        """
        
        self.time = 0
        self.metric = 0
        
        diagram, plant, scene_graph = MakeIiwaPaddleStation(time_step=time_step)
        
        self.plant = plant
        self.builder = DiagramBuilder()
        self.station = self.builder.AddSystem(diagram)
        
        self.visualizer = MeshcatVisualizerCpp.AddToBuilder(
            self.builder,
            diagram.GetOutputPort("geometry_query"),
            meshcat,
            MeshcatVisualizerParams(delete_prefix_initialization_event=False)
        )

        self.plant.mutable_gravity_field().set_gravity_vector([0, 0, g])

        ik_sys = self.builder.AddSystem(PseudoInverseController(self.plant))
        ik_sys.set_name("ik_sys")
        catcher = self.builder.AddSystem(Catcher(self.plant))
        catcher.set_name("catcher")
        integrator = self.builder.AddSystem(Integrator(7))
        integrator.set_name("integrator")

        self.builder.Connect(ik_sys.get_output_port(), integrator.get_input_port())
        self.builder.Connect(integrator.get_output_port(), self.station.GetInputPort("iiwa_position"))
        self.builder.Connect(self.station.GetOutputPort("ball_state"), catcher.GetInputPort("ball_state"))
        self.builder.Connect(self.station.GetOutputPort("iiwa_position_measured"), ik_sys.GetInputPort("iiwa_pos_measured"))
        self.builder.Connect(self.station.GetOutputPort("iiwa_position_measured"), catcher.GetInputPort("iiwa_pos_measured"))
        self.builder.Connect(catcher.GetOutputPort("paddle_desired_velocity"), ik_sys.GetInputPort("paddle_desired_velocity"))
        self.builder.Connect(self.station.GetOutputPort("iiwa_velocity_estimated"), catcher.GetInputPort("iiwa_velocity_estimated"))
        self.builder.Connect(catcher.GetOutputPort("paddle_desired_angular_velocity"), ik_sys.GetInputPort("paddle_desired_angular_velocity"))
        
        self.diagram = self.builder.Build()
        self.simulator = Simulator(self.diagram)
        self.simulator.set_target_realtime_rate(1.0)

        self.context = self.simulator.get_context()
        self.station_context = self.station.GetMyContextFromRoot(self.context)
        self.plant_context = self.plant.GetMyContextFromRoot(self.context)

        self.plant.SetPositions(self.plant_context, self.plant.GetModelInstanceByName("iiwa7"), np.array(iiwa_q0))
        self.plant.SetPositionsAndVelocities(self.plant_context, self.plant.GetModelInstanceByName("ball"), np.array(scenarios[exp_idx]))

        self.station.GetInputPort("iiwa_feedforward_torque").FixValue(self.station_context, np.zeros((7,1)))
        iiwa_q = self.plant.GetPositions(self.plant_context, self.plant.GetModelInstanceByName("iiwa7"))
        integrator.GetMyContextFromRoot(self.context).get_mutable_continuous_state_vector().SetFromVector(iiwa_q)

    def step(self, simulate=True, duration=0.1, final=True):
        """
        Step the closed loop system

        Args:
            simulate (bool, optional): whether or not to visualize the command. Defaults to True.
            duration (float, optional): duration to complete command in simulation. Defaults to 0.1.
            final (bool, optional): whether or not this is the final command in the sequence; relevant for recording. Defaults to True.
        """
        if simulate:

            self.visualizer.StartRecording()
            self.simulator.AdvanceTo(self.time + duration)
            self.visualizer.StopRecording()

            # Compute the stability metric
            contact_results = self.plant.get_contact_results_output_port().Eval(self.plant_context)
            for i in range(contact_results.num_point_pair_contacts()):
                info = contact_results.point_pair_contact_info(i)
                bodyA = self.plant.get_body(info.bodyA_index()).name()
                bodyB = self.plant.get_body(info.bodyB_index()).name()
                if (bodyA == "ball" and bodyB == "base_link") or (bodyA == "base_link" and bodyB == "ball"):
                    self.metric += duration

            if final:
                self.visualizer.PublishRecording()
                return self.time + duration

        self.time += duration

### Experiments

In [ ]:
time_step = .0005

for idx, experiment in enumerate(scenarios):
    
    start_time = time.perf_counter()
    demo = Demo(time_step=time_step, exp_idx=idx)

    seconds = simulation_time
    for i in range(int(seconds * 20)):
        t = demo.step(duration=0.05, final=i==seconds * 20 - 1)
        if t is not None and t != seconds:
            break
    end_time = time.perf_counter()

    metric_val = demo.metric
    print("Experiment: ", idx)
    print("Stability Metric: ", metric_val)
    print("Total Time(s): ", (end_time - start_time))

Experiment:  0
Stability Metric:  2.3499999999999996
Total Time(s):  13.677109789976384
Experiment:  1
Stability Metric:  0
Total Time(s):  14.37338693201309
Experiment:  2
Stability Metric:  0
Total Time(s):  13.990958713984583
Experiment:  3
Stability Metric:  2.3999999999999995
Total Time(s):  13.84157278802013
Experiment:  4
Stability Metric:  0
Total Time(s):  12.952422180038411
Experiment:  5
Stability Metric:  0
Total Time(s):  13.638244081987068
Experiment:  6
Stability Metric:  0
Total Time(s):  14.251578803989105
Experiment:  7
Stability Metric:  0
Total Time(s):  14.187160459987354
Experiment:  8
Stability Metric:  0
Total Time(s):  14.149227285990492


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=b95b60bf-1e32-44f8-a07c-9240db62d20a' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>